# Amazon Bedrock AgentCore Policy - Lambda 타겟 설정

## 개요

이 노트북은 **Lambda 타겟**과 함께 **Amazon Bedrock AgentCore Policy**를 테스트하기 위한 인프라를 설정합니다.

### 이 노트북에서 생성하는 리소스

| 리소스 | 설명 |
|--------|------|
| Lambda 함수 | `RefundLambda` - 샘플 환불 처리 함수 |
| AgentCore Gateway | JWT Authorizer가 포함된 MCP 프로토콜 엔드포인트 |
| Gateway 타겟 | Gateway에 연결된 Lambda 도구 |
| Cognito User Pool | 인증 컨테이너 |
| Cognito App Client | 토큰 요청을 위한 M2M 클라이언트 |
| Cognito Resource Server | OAuth2 scope 정의 |

### 아키텍처

```
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────┐
│   Amazon        │  JWT 토큰   │  AgentCore Gateway    │             │   Lambda    │
│   Cognito       │────────────>│  + JWT Authorizer     │────────────>│   타겟      │
│                 │             │                       │             │   (도구)    │
└─────────────────┘             └───────────────────────┘             └─────────────┘
```

> **다음 단계**: 이 노트북을 완료한 후, `02-Policy-Enforcement.ipynb`를 실행하여 Cedar 정책을 테스트하세요.

---

## Part 1: 환경 설정

### 가상환경 설정 (권장)

```bash
cd ../00_setup
chmod +x create_uv_virtual_env.sh
./create_uv_virtual_env.sh AgentCorePolicy
```

Jupyter에서 `AgentCorePolicy` 커널을 선택하세요.

In [ ]:
import json
import sys
import time
from pathlib import Path

import boto3

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions from common folder
from common.auth_utils import (
    get_bearer_token,
    decode_token,
    make_gateway_request,
    analyze_response,
)
from common.gateway_utils import (
    validate_and_fix_gateway_authorizer,
)

print("✓ Libraries loaded")

---

## Part 2: Gateway 및 Cognito 생성

`setup-gateway.py`를 실행하여 필요한 모든 AWS 리소스를 생성합니다.

> ⚠️ `gateway_config.json`이 이미 존재하면 이 셀을 건너뛰세요.

In [ ]:
# Create Gateway and Cognito (only if gateway_config.json doesn't exist)
config_file = Path.cwd() / "gateway_config.json"

if not config_file.exists():
    print("gateway_config.json not found. Creating Gateway and Cognito...")
    %run setup-gateway.py --region us-east-1
else:
    print(f"✓ gateway_config.json already exists: {config_file}")

### 생성된 리소스

#### AgentCore Gateway
![Gateway.png](img/Gateway.png)

#### Lambda 함수
![refund_lambda.png](img/refund_lambda.png)

#### Cognito User Pool
![cognito_user_pool](img/cognito_user_pool.png)

#### Cognito App Client
![Cognito_App_Clients.png](img/Cognito_App_Clients.png)

#### Cognito 도메인
![Cognito_Domain.png](img/Cognito_Domain.png)

#### Cognito Resource Server
![Cognito_Resource_Server.png](img/Cognito_Resource_Server.png)

---

## Part 3: 설정 로드

In [ ]:
# Load configuration file
config_path = Path.cwd() / "gateway_config.json"

with open(config_path, "r") as f:
    CONFIG = json.load(f)

# Extract key values
REGION = CONFIG["region"]
GATEWAY_URL = CONFIG["gateway_url"]
GATEWAY_ID = CONFIG["gateway_id"]
GATEWAY_ARN = CONFIG["gateway_arn"]
USER_POOL_ID = CONFIG["client_info"]["user_pool_id"]
CLIENT_ID = CONFIG["client_info"]["client_id"]
CLIENT_SECRET = CONFIG["client_info"]["client_secret"]
TOKEN_ENDPOINT = CONFIG["client_info"]["token_endpoint"]

print("✓ Configuration loaded")
print(f"  REGION: {REGION}")
print(f"  GATEWAY_ID: {GATEWAY_ID}")
print(f"  GATEWAY_URL: {GATEWAY_URL}")
print(f"  GATEWAY_ARN: {GATEWAY_ARN}")
print(f"  USER_POOL_ID: {USER_POOL_ID}")
print(f"  CLIENT_ID: {CLIENT_ID}")
print(f"  TOKEN_ENDPOINT: {TOKEN_ENDPOINT}")

---

## Part 4: AWS 클라이언트 초기화

In [ ]:
session = boto3.Session(region_name=REGION)

sts_client = session.client("sts")
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)

ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")
print(f"  Region: {REGION}")

---

## Part 5: Gateway Authorizer 검증

Gateway JWT Authorizer가 Cognito에 맞게 올바르게 설정되어 있는지 확인합니다.

In [ ]:
# Validate and fix Gateway Authorizer if needed
scope = CONFIG.get("client_info", {}).get("scope", "")
print(f"Scope: {scope}")

validate_and_fix_gateway_authorizer(
    gateway_control_client=gateway_control_client,
    gateway_id=GATEWAY_ID,
    region=REGION,
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    scope=scope
)

---

## Part 6: 설정 확인

Gateway와 Lambda 타겟이 올바르게 작동하는지 테스트합니다 (정책 적용 없이).

In [ ]:
# Get Bearer Token
print("Getting Bearer Token...")
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

print(f"✓ Token received")
print(f"  Token (first 50 chars): {token[:50]}...")

# Decode and show claims
claims = decode_token(token)
print(f"\n🔍 Token Claims:")
print(f"  client_id: {claims.get('client_id', 'N/A')}")
print(f"  scope: {claims.get('scope', 'N/A')}")

In [ ]:
# Test Gateway request (without policy - should work or fail based on current policy state)
print("\nTesting Gateway request...")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 100, "orderId": "test-setup-verification"}
)

print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
print(f"\nResult: {outcome}")

if outcome == "ALLOWED":
    print("\n✓ Setup verification successful! Gateway and Lambda are working.")
elif outcome == "DENIED":
    print("\n⚠️  Request was denied by policy. This is expected if policies are attached.")
    print("   Run 02-Policy-Enforcement.ipynb to configure and test policies.")
else:
    print(f"\n⚠️  Unexpected result: {outcome}")
    print("   Check the response above for details.")

---

## 요약

이 노트북에서 설정한 내용:

✅ Lambda 함수 (`RefundLambda`)  
✅ JWT Authorizer가 포함된 AgentCore Gateway  
✅ Gateway 타겟 (Lambda 도구)  
✅ Cognito User Pool, App Client, Resource Server  
✅ Gateway Authorizer 설정 검증  

### 다음 단계

**`02-Policy-Enforcement.ipynb`**를 실행하여:
- Policy Engine 생성 및 Gateway에 연결
- 세분화된 접근 제어를 위한 Cedar 정책 테스트
- 부서 기반 및 그룹 기반 접근 패턴 학습